In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy.optimize import curve_fit

In [ ]:
df = pd.read_csv(
    '../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv',
    usecols=['t_dat', 'customer_id', 'article_id'])

df['t_dat'] = pd.to_datetime(df['t_dat'])

In [ ]:
gb = df.groupby(['customer_id', 'article_id'])['t_dat'].nunique()
inx = gb[gb>1].index

In [ ]:
df = df.set_index(['customer_id', 'article_id'])
df = df.loc[inx].copy().sort_index()

In [ ]:
dist = []

for i in tqdm(inx):
    t_prev = None
    for t_dat in df.loc[i, 't_dat']:
        if t_prev is not None:
            dist.append((t_dat - t_prev).days)
        t_prev = t_dat

In [ ]:
vc = pd.Series(dist).value_counts()
vc = vc[1:]

In [ ]:
def func(x, a, b, c, d):
    return a / np.sqrt(x) + b * np.exp(-c*x) + d

popt, pcov = curve_fit(func, vc.index, vc.values)

plt.plot(vc.index[:100], vc.values[:100], 'b-', label='data')
plt.plot(vc.index[:100], func(vc.index[:100], *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f, d=%5.3f' % tuple(popt))

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()